In [1]:
#configuración de Spark en Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#configuración de Spark en Google Colab
#instalar java y spark
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz
!pip install -q findspark

In [3]:
#configuración de Spark en Google Colab
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"

In [4]:
#configuración de Spark en Google Colab
import findspark
findspark.init()

In [5]:
#configuración de Spark en Google Colab
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [6]:
# WORDCOUNT COMPACTO
# en AWS S3
#files_rdd = sc.textFile("s3a://username_datalake/datasets/gutenberg-small/*.txt")

# en gdrive:
files_rdd = sc.textFile("/content/drive/MyDrive/data/gutenberg-small/*")

# local:
#files_rdd = sc.textFile("../datasets/gutenberg-small/*.txt")
wc_unsort = files_rdd.flatMap(lambda line: line.split()).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
wc = wc_unsort.sortBy(lambda a: -a[1])
for tupla in wc.take(10):
        print(tupla)

('the', 44647)
('of', 28020)
('to', 23208)
('and', 20444)
('in', 13174)
('that', 12265)
('I', 10880)
('a', 10431)
('is', 7776)
('be', 7148)


In [ ]:
# WORDCOUNT PASO A PASO

In [7]:
# en AWS S3
#files = sc.textFile("s3a://username_datalake/datasets/gutenberg-small/*.txt")
files = sc.textFile("/content/drive/MyDrive/data/gutenberg-small/*")
for f in files.take(10):
    print(f)


LINCOLN LETTERS

By Abraham Lincoln


Published by The Bibilophile Society





In [8]:
tokens = files.flatMap(lambda line: line.split())
for t in tokens.take(10):
    print(t)

LINCOLN
LETTERS
By
Abraham
Lincoln
Published
by
The
Bibilophile
Society


In [9]:
wc1 = tokens.map(lambda word: (word, 1))
for c in wc1.take(10):
    print(c)

('LINCOLN', 1)
('LETTERS', 1)
('By', 1)
('Abraham', 1)
('Lincoln', 1)
('Published', 1)
('by', 1)
('The', 1)
('Bibilophile', 1)
('Society', 1)


In [10]:
wc = wc1.reduceByKey(lambda a, b: a + b)
for c in wc.take(10):
    print(c)

('Published', 3)
('themselves', 192)
('were', 1450)
('sheet', 4)
('despatched', 4)
('most', 551)
('turbulent', 2)
('A.', 1456)
('ORIGINALS', 1)
('IN', 84)


In [11]:
wcsort = wc.sortBy(lambda a: -a[1])
for c in wcsort.take(10):
    print(c)

('the', 44647)
('of', 28020)
('to', 23208)
('and', 20444)
('in', 13174)
('that', 12265)
('I', 10880)
('a', 10431)
('is', 7776)
('be', 7148)


In [12]:
wc.saveAsTextFile("/content/drive/MyDrive/data/gutenberg-small/output-wordcount")